<a href="https://colab.research.google.com/github/JiaUF/OnlineQuantumLab/blob/main/Deutsch_Jozsa_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Before running the codes, please install qiskit in the Colab terminal by using the commands:  
> pip install qiskit  
> pip install 'qiskit[visualization]'  
> pip install qiskit-aer


# Deutsch-Jozsa Algorithm: Implementation in Qiskit

This notebook walks you through the Deutsch-Jozsa algorithm for determining whether a function $f(x)$ is constant or balanced.

In the Deutsch–Jozsa algorithm, we are given a special type of function $f(x)$ that takes a 1-bit input (either 0 or 1) and outputs either 0 or 1. We are promised that the function is one of the following:

- **Constant function**:
  The output is the same for all inputs. That is, $f(0) = f(1)$.
  Examples:
  - $f(0) = 0,\ f(1) = 0$
  - $f(0) = 1,\ f(1) = 1$

- **Balanced function**:
  The output is 0 for one input and 1 for the other. That is, $f(0) \ne f(1)$.
  Examples:
  - $f(0) = 0,\ f(1) = 1$
  - $f(0) = 1,\ f(1) = 0$

The goal of the Deutsch–Jozsa algorithm is to determine whether the unknown function is **constant** or **balanced**, using just **one** evaluation on a quantum computer. This highlights the power of quantum algorithms compared to classical ones.

The key idea is to use **quantum parallelism and interference** to determine the function type with just one query.

--------

## Task 1: single-qubit Deutsch-Jozsa algorithm
Now, let's explore how to implement the Deutsch-Jozsa algorithm using the Qiskit quantum simulator.



In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

### Step 1: Define the Oracle for the function $f(x)$

The oracle will implement the transformation:

$|x⟩|y⟩ \rightarrow |x⟩|y \oplus f(x)⟩$

We create oracles for constant and balanced functions.


In [ ]:
# Return a quantum circuit implementing the DJ oracle for f(x).
# You can understand oracle as a black box that performs some computation.

def deutsch_jozsa_oracle(case='constant'):
    # We create a quantum circuit that has 2 qubits.
    oracle = QuantumCircuit(2)
    if case == 'balanced':
        # control-NOT gate. it applies a NOT (X) gate to the second qubit
        # (index 1) if and only if the first qubit (index 0) is in the state |1⟩
        oracle.cx(0, 1)
    elif case == 'constant':
        pass  # identity if f(x) = 0
    return oracle


### Step 2: Deutsch-Jozsa Circuit Construction

We initialize the qubits to $|0⟩$ and $|1⟩$, then apply Hadamard gates to both, apply the oracle, and apply another Hadamard to the first qubit.


In [ ]:
def deutsch_jozsa_circuit(case='constant'):
    qc = QuantumCircuit(2, 1)

    # Initialize |0⟩|1⟩
    qc.x(1)  # Set bottom qubit to |1⟩
    qc.h([0, 1])  # Apply Hadamard to both

    # Apply the oracle
    oracle = deutsch_jozsa_oracle(case)
    qc.append(oracle.to_gate(), [0, 1])

    # Apply Hadamard to the first qubit
    qc.h(0)

    # Measure the first qubit
    qc.measure(0, 0)
    print(qc)

    return qc

### Step 3: Run the circuit and interpret results

If the result is always 0, then the function is **constant**.  
If the result is 1, then the function is **balanced**.


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def run_and_plot(case='constant'):
    # Get Deutsch-Jozsa circuit
    qc = deutsch_jozsa_circuit(case)

    # Choose backend simulator
    backend = Aer.get_backend('aer_simulator')

    # Transpile circuit for backend
    transpiled = transpile(qc, backend)

    # Run with memory enabled to access individual shots
    job = backend.run(transpiled, shots=1024, memory=True)
    result = job.result()

    # Access full shot memory
    memory = result.get_memory()
    print(f"[{case.upper()}] First 5 measurement outcomes:", memory[:5])

    # Access and plot measurement counts
    counts = result.get_counts()
    print(f"[{case.upper()}] Measurement counts:", counts)
    #plot_histogram(counts)
    #plt.title(f"Deutsch-Jozsa result for a {case} function")
    #plt.show()

# Run both test cases
run_and_plot('constant')
run_and_plot('balanced')

     ┌───┐     ┌──────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0             ├┤ H ├┤M├
     ├───┤┌───┐│  circuit-212 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1             ├──────╫─
     └───┘└───┘└──────────────┘      ║ 
c: 1/════════════════════════════════╩═
                                     0 
[CONSTANT] First 5 measurement outcomes: ['0', '0', '0', '0', '0']
[CONSTANT] Measurement counts: {'0': 1024}
     ┌───┐     ┌──────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0             ├┤ H ├┤M├
     ├───┤┌───┐│  circuit-217 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1             ├──────╫─
     └───┘└───┘└──────────────┘      ║ 
c: 1/════════════════════════════════╩═
                                     0 
[BALANCED] First 5 measurement outcomes: ['1', '1', '1', '1', '1']
[BALANCED] Measurement counts: {'1': 1024}


### Interpreting the Results for Task 1

As you can see from the results, when the function $f(x)$ is **constant**, the measurement of qubit 0 always yields **0**.  
In contrast, when $f(x)$ is **balanced**, the measurement of qubit 0 always yields **1**.  

This clear distinction is what makes the Deutsch–Jozsa algorithm powerful—it determines the global property of a function (constant or balanced) with a single query.

---

## Task 2: Multi-qubit Deutsch-Jozsa Algorithm

This version generalizes the one-qubit Deutsch-Jozsa algorithm to the multi-qubit case and demonstrates how quantum parallelism and interference can be harnessed to solve certain problems exponentially faster than classical algorithms.

In the multi-qubit version, the algorithm uses n input qubits and 1 output qubit to evaluate a function $f(x)$. The goal is to determine whether $f(x)$ is constant (same output for all inputs) or balanced (outputs 0 for half the inputs and 1 for the other half).

This quantum algorithm accomplishes this with a **single oracle query**, embedded within a circuit composed of a fixed number of quantum gates. This allows us to determine with certainty whether $f(x)$ is constant or balanced. In contrast, any classical algorithm would require up to $2^{n-1} + 1$ evaluations in the worst case.

In [25]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import random

def deutsch_jozsa_oracle(n, case='constant'):
    """
    Create a Deutsch-Jozsa oracle for an n-bit function.
    'constant' returns all 0 or all 1; 'balanced' flips output for half the inputs.
    """
    oracle = QuantumCircuit(n + 1)

    if case == 'constant':
        if random.choice([True, False]):
            oracle.x(n)  # flip output qubit to simulate f(x) = 1
        # else do nothing (f(x) = 0)

    elif case == 'balanced':
        for i in range(n):
            if random.choice([True, False]):
                oracle.cx(i, n)

    return oracle

def deutsch_jozsa_circuit(n, case='constant'):
    """
    Build the Deutsch–Jozsa algorithm circuit for n input qubits.
    """
    qc = QuantumCircuit(n + 1, n)

    # Initialize output qubit to |1⟩ and apply Hadamard
    qc.x(n)
    qc.h(n)

    # Apply Hadamard to all input qubits
    for i in range(n):
        qc.h(i)

    # Append the oracle
    oracle = deutsch_jozsa_oracle(n, case)
    qc.append(oracle.to_gate(), list(range(n + 1)))

    # Apply Hadamard again to all input qubits
    for i in range(n):
        qc.h(i)

    # Measure input qubits
    for i in range(n):
        qc.measure(i, i)
    print(qc)
    return qc

def run_deutsch_jozsa(n, case='constant'):
    qc = deutsch_jozsa_circuit(n, case)
    backend = Aer.get_backend('aer_simulator')
    transpiled = transpile(qc, backend)
    result = backend.run(transpiled, shots=1024).result()
    counts = result.get_counts()

    print(f"\nResults for a {case.upper()} function with {n} qubits:")
    print(counts)
    #plot_histogram(counts)
    #plt.title(f"Deutsch–Jozsa: {case.capitalize()} Function")
    #plt.show()

# Example usage
run_deutsch_jozsa(n=6, case='constant')
run_deutsch_jozsa(n=6, case='balanced')

     ┌───┐     ┌──────────────┐┌───┐┌─┐               
q_0: ┤ H ├─────┤0             ├┤ H ├┤M├───────────────
     ├───┤     │              │├───┤└╥┘┌─┐            
q_1: ┤ H ├─────┤1             ├┤ H ├─╫─┤M├────────────
     ├───┤     │              │├───┤ ║ └╥┘┌─┐         
q_2: ┤ H ├─────┤2             ├┤ H ├─╫──╫─┤M├─────────
     ├───┤     │              │├───┤ ║  ║ └╥┘┌─┐      
q_3: ┤ H ├─────┤3 circuit-242 ├┤ H ├─╫──╫──╫─┤M├──────
     ├───┤     │              │├───┤ ║  ║  ║ └╥┘┌─┐   
q_4: ┤ H ├─────┤4             ├┤ H ├─╫──╫──╫──╫─┤M├───
     ├───┤     │              │├───┤ ║  ║  ║  ║ └╥┘┌─┐
q_5: ┤ H ├─────┤5             ├┤ H ├─╫──╫──╫──╫──╫─┤M├
     ├───┤┌───┐│              │└───┘ ║  ║  ║  ║  ║ └╥┘
q_6: ┤ X ├┤ H ├┤6             ├──────╫──╫──╫──╫──╫──╫─
     └───┘└───┘└──────────────┘      ║  ║  ║  ║  ║  ║ 
c: 6/════════════════════════════════╩══╩══╩══╩══╩══╩═
                                     0  1  2  3  4  5 

Results for a CONSTANT function with 6 qubits:
{'000000': 1024}


### Interpreting the Results for Task 2

If the function is constant, the measurement outcome is always the all-zeros bitstring (e.g., '000'). If the function is balanced, the result includes one or more nonzero bitstrings, depending on the specific oracle used. This demonstrates how the Deutsch-Jozsa algorithm can efficiently distinguish between constant and balanced functions in the multi-qubit case, using just one evaluation of the quantum oracle (we only call the function $f(x)$ once).